<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting" data-toc-modified-id="Setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting</a></span></li><li><span><a href="#Gene-Meta" data-toc-modified-id="Gene-Meta-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gene Meta</a></span></li><li><span><a href="#Cell-Meta" data-toc-modified-id="Cell-Meta-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cell Meta</a></span><ul class="toc-item"><li><span><a href="#Determine-meta-cols" data-toc-modified-id="Determine-meta-cols-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Determine meta cols</a></span></li><li><span><a href="#Select-meta-cols" data-toc-modified-id="Select-meta-cols-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Select meta cols</a></span></li></ul></li><li><span><a href="#Coords" data-toc-modified-id="Coords-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Coords</a></span><ul class="toc-item"><li><span><a href="#Clustering-results" data-toc-modified-id="Clustering-results-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Clustering results</a></span></li><li><span><a href="#Dissection-Region-embedding" data-toc-modified-id="Dissection-Region-embedding-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Dissection Region embedding</a></span></li></ul></li><li><span><a href="#Gene-Matrix" data-toc-modified-id="Gene-Matrix-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Gene Matrix</a></span><ul class="toc-item"><li><span><a href="#Save-gene-matrix" data-toc-modified-id="Save-gene-matrix-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Save gene matrix</a></span></li></ul></li><li><span><a href="#Conf-file" data-toc-modified-id="Conf-file-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conf file</a></span></li></ul></div>

In [1]:
import pathlib
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
page_name = 'CEMBA-RostralBrain'
mc_type = 'CGN'

result_dir = pathlib.Path(f'{page_name}_{mc_type}').absolute()
result_dir.mkdir(exist_ok=True)

## Setting

In [3]:
cell_meta_path = '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'

gene_mcds_path_pattern = '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/SelectedCell*gene_da_rate*mcds'
gene_annotation_path = '/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz'

COLOR_PATH = '/home/hanliu/project/mouse_rostral_brain/metadata/palette/total_meta_colors.tsv'

quick_gene_path = '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/CellBrowser/quickGenes.csv'

# additional GOI that must include in browser
goi_path = None
MUST_INCLUDE_GENE_TYPE = ['protein_coding', 'lncRNA']

CELL_META_PATH = result_dir / f'{page_name}.cell_meta.tsv'

default_cluster_col = 'MajorType'
default_label_col = 'MajorType'

radius = 2.5

## Gene Meta

In [4]:
gene_meta = pd.read_csv(gene_annotation_path, index_col='gene_id', sep='\t')
gene_meta.index.name = 'gene'
gene_name_to_id = {v:k for k, v in gene_meta['gene_name'].iteritems()}
gene_id_base_to_id = pd.Series(gene_meta.index, index = gene_meta.index.map(lambda i: i.split('.')[0])).to_dict()

# select gene to include in browser
# keep all gene in MUST_INCLUDE_GENE_TYPE
# keep level 1 exp validated gene if not in MUST_INCLUDE_GENE_TYPE
gene_meta = gene_meta[(gene_meta['level'] == 1) | gene_meta['gene_type'].isin(MUST_INCLUDE_GENE_TYPE)].copy()
gene_meta.shape[0]

40836

## Cell Meta

In [5]:
cell_meta = pd.read_msgpack(cell_meta_path)
print(cell_meta.shape)
print(cell_meta.columns.tolist())
cell_meta.head()

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


(103982, 39)
['AllcPath', 'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate', 'CH_RateAdj', 'FinalReads', 'InputReads', 'MappedReads', 'Region', 'index_name', 'uid', 'BamFilteringRate', 'MappingRate', 'Pos96', 'Plate', 'Col96', 'Row96', 'Col384', 'Row384', 'FACS_Date', 'Slice', 'MajorRegion', 'SubRegion', 'CellClass', 'l1-umap_0', 'l1-umap_1', 'l1-tsne_0', 'l1-tsne_1', 'MajorType', 'l2-umap_0', 'l2-umap_1', 'l2-tsne_0', 'l2-tsne_1', 'SubType', 'l3-umap_0', 'l3-umap_1', 'l3-tsne_0', 'l3-tsne_1']


,AllcPath,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,MajorType,l2-umap_0,l2-umap_1,l2-tsne_0,l2-tsne_1,SubType,l3-umap_0,l3-umap_1,l3-tsne_0,l3-tsne_1
10E_M_0,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.008198,0.822633,0.821166,0.041640,0.033718,1626504.0,4407752,2892347.0,10E,...,MGE-Sst,-23.368460,-1.151200,-34.021153,-19.429447,MGE-Sst Rxra,-0.766630,-16.704815,20.297781,1.192093
10E_M_1,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006019,0.743035,0.741479,0.024127,0.018218,2009998.0,5524084,3657352.0,10E,...,CA3,-8.322600,-3.988057,19.205085,10.272570,CA3 Cadm2,4.290240,-2.610511,7.860870,8.813396
10E_M_10,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006569,0.750172,0.748520,0.027665,0.021235,1383636.0,3455260,2172987.0,10E,...,CA3,-7.409779,-4.662556,20.146606,6.345836,CA3 Cadm2,5.852028,-2.510058,12.927795,8.544050
10E_M_101,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006353,0.760898,0.759369,0.026547,0.020323,2474670.0,7245482,4778768.0,10E,...,CA3,-6.266342,-8.345406,25.465765,-6.767192,CA3 Cadm2,11.189178,-3.763147,29.875836,2.739283
10E_M_102,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.005409,0.752980,0.751637,0.019497,0.014164,2430290.0,7004754,4609570.0,10E,...,CA1,-1.135772,-17.311239,-7.427455,-20.497435,CA1 Chrm3,2.912797,2.683262,-19.971042,24.551260


### Determine meta cols

In [6]:
exclude_cols = ['AllcPath']

categorical_cols = ['Row384', 'Col384', 'Row96', 'Col96', 'Pos96', 
                    'FACS_Date', 'Plate', 'Region', 
                    'Slice', 'index_name', 'uid', 
                    'CellClass', 'MajorType', 'SubType'
                    'MajorRegion', 'SubRegion']

### Select meta cols

In [7]:
# save the non-coords part
cell_meta_no_coords = cell_meta.loc[:, (~(cell_meta.columns.str.endswith('_0') | 
                                         cell_meta.columns.str.endswith('_1'))) & 
                                       (~cell_meta.columns.isin(exclude_cols))].copy()

In [8]:
cell_meta_no_coords.index.name = 'Cell'

CELL_META_PATH = result_dir / f'{page_name}.cell_meta.tsv'
cell_meta_no_coords.to_csv(CELL_META_PATH, sep='\t')

## Coords

### Clustering results

In [9]:
# save the non-coords part
cell_meta_coords = cell_meta.loc[:, (cell_meta.columns.str.endswith('_0') | 
                                     cell_meta.columns.str.endswith('_1'))].copy()

In [10]:
cell_meta.columns

Index(['AllcPath', 'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate',
       'CH_RateAdj', 'FinalReads', 'InputReads', 'MappedReads', 'Region',
       'index_name', 'uid', 'BamFilteringRate', 'MappingRate', 'Pos96',
       'Plate', 'Col96', 'Row96', 'Col384', 'Row384', 'FACS_Date', 'Slice',
       'MajorRegion', 'SubRegion', 'CellClass', 'l1-umap_0', 'l1-umap_1',
       'l1-tsne_0', 'l1-tsne_1', 'MajorType', 'l2-umap_0', 'l2-umap_1',
       'l2-tsne_0', 'l2-tsne_1', 'SubType', 'l3-umap_0', 'l3-umap_1',
       'l3-tsne_0', 'l3-tsne_1'],
      dtype='object')

In [11]:
COORD_PATH_LIST = []

for coord_base, coord_df in cell_meta_coords.groupby(cell_meta_coords.columns.str[:-2], axis=1):
    print(coord_base.upper(), 'included in browser')
    coord_df = coord_df[[f'{coord_base}_0', f'{coord_base}_1']].copy()
    coord_df.index.name = 'cellId'
    coord_df.columns = ['x', 'y']
    if coord_base.upper().startswith('L2'):
        # save each cell class separately
        
        # exclude outlier
        selected_cell = cell_meta.index[cell_meta['MajorType'].apply(lambda i: 'Outlier' not in i)]
        _coord_df = coord_df.reindex(selected_cell)
        
        for cell_class, sub_df in _coord_df.groupby(cell_meta['CellClass']):
            print(coord_base, cell_class)
            # cellBrowser only support 2D
            coord_path = result_dir / f'{page_name}.{coord_base}.{cell_class}.tsv'
            sub_df.to_csv(coord_path, sep='\t')
            COORD_PATH_LIST.append({'file': str(coord_path), 
                                    'shortLabel': f'{coord_base.upper()} {cell_class}'})
    elif coord_base.upper().startswith('L3'):
        # exclude outlier
        selected_cell = cell_meta.index[cell_meta['SubType'].apply(lambda i: 'Outlier' not in i)]
        _coord_df = coord_df.reindex(selected_cell)
            
        for cell_class, sub_df in _coord_df.groupby(cell_meta['MajorType']):
            print(coord_base, cell_class)
            
            # cellBrowser only support 2D
            coord_path = result_dir / f'{page_name}.{coord_base}.{cell_class}.tsv'
            sub_df.to_csv(coord_path, sep='\t')
            COORD_PATH_LIST.append({'file': str(coord_path), 
                                    'shortLabel': f'{coord_base.upper()} {cell_class}',
                                    'colorOnMeta': "SubType"})
    else:
        # cellBrowser only support 2D
        coord_path = result_dir / f'{page_name}.{coord_base}.tsv'
        coord_df.to_csv(coord_path, sep='\t')
        COORD_PATH_LIST.append({'file': str(coord_path), 
                                'shortLabel': f'{coord_base.upper()} ALL',
                                'colorOnMeta': "SubRegion"})


L1-TSNE included in browser
L1-UMAP included in browser
L2-TSNE included in browser
l2-tsne Exc
l2-tsne Exc-HPF
l2-tsne GC
l2-tsne Inh
l2-tsne NonN
L2-UMAP included in browser
l2-umap Exc
l2-umap Exc-HPF
l2-umap GC
l2-umap Inh
l2-umap NonN
L3-TSNE included in browser
l3-tsne ANP
l3-tsne ASC
l3-tsne CA1
l3-tsne CA3
l3-tsne CA3-St18
l3-tsne CGE-Lamp5
l3-tsne CGE-Vip
l3-tsne CLA
l3-tsne CT-L6
l3-tsne Chd7
l3-tsne D1L-Fstl4
l3-tsne D1L-PAL
l3-tsne DG
l3-tsne DG-po
l3-tsne EC
l3-tsne EP
l3-tsne Foxp2
l3-tsne Gfra1
l3-tsne IG-CA2
l3-tsne IT-L23
l3-tsne IT-L4
l3-tsne IT-L5
l3-tsne IT-L6
l3-tsne L6b
l3-tsne LSX-Inh
l3-tsne MGC
l3-tsne MGE-Pvalb
l3-tsne MGE-Sst
l3-tsne MSN-D1
l3-tsne MSN-D2
l3-tsne NP-L6
l3-tsne ODC
l3-tsne OLF
l3-tsne OLF-Exc
l3-tsne OPC
l3-tsne PAL-Inh
l3-tsne PC
l3-tsne PT-L5
l3-tsne Unc5c
l3-tsne VLMC
l3-tsne VLMC-Pia
L3-UMAP included in browser
l3-umap ANP
l3-umap ASC
l3-umap CA1
l3-umap CA3
l3-umap CA3-St18
l3-umap CGE-Lamp5
l3-umap CGE-Vip
l3-umap CLA
l3-umap CT-L6
l3-um

### Dissection Region embedding

In [12]:
dissection_embedding_dir = pathlib.Path('/home/hanliu/project/mouse_rostral_brain/study/DissectionRegions/')
dissection_results = pd.concat([pd.read_msgpack(i) 
                                for i in dissection_embedding_dir.glob('*/Coords/cell_tidy_data.msg')])

use_cells = cell_meta.index[cell_meta['MajorType'].apply(lambda i: 'Outlier' not in i)]
dissection_results = dissection_results.reindex(use_cells)
dissection_results.index.name = 'cellId'

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
dissection_coords_list = []
for dissection_region, sub_df in dissection_results.groupby('Region'):
    for coord_base in ['umap', 'tsne']:
        coord_df = sub_df[[f'{coord_base}_0', f'{coord_base}_1']].copy()
        coord_df.columns = ['x', 'y']
        coord_path = result_dir / f'{page_name}.{coord_base}.REGION_{dissection_region}.tsv'
        coord_df.to_csv(coord_path, sep='\t')
        dissection_coords_list.append({'file': str(coord_path), 
                                       'shortLabel': f'CEMBA {dissection_region} {coord_base.upper()}',
                                       'colorOnMeta': "SubType"})

In [14]:
COORD_PATH_LIST = COORD_PATH_LIST + dissection_coords_list

## Gene Matrix

In [15]:
from ALLCools.mcds.MCDS import MCDS

gene_mcds = MCDS.open(gene_mcds_path_pattern,
                      combine='nested',
                      concat_dim='cell').sel(mc_type=mc_type, cell=cell_meta.index)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/merge.py:18: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1829: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  'DataArray', pd.Series, pd.DataFrame, pd.Panel]:


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
gene_mcds

<xarray.MCDS>
Dimensions:      (cell: 103982, gene: 55487)
Coordinates:
    mc_type      <U3 'CGN'
  * gene         (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
    strand_type  <U4 'both'
    gene_chrom   (gene) object dask.array<shape=(55487,), chunksize=(55487,)>
    gene_start   (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
    gene_end     (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
  * cell         (cell) object '10E_M_0' '10E_M_1' ... '9J_M_998' '9J_M_999'
Data variables:
    gene_da      (cell, gene) float64 dask.array<shape=(103982, 55487), chunksize=(456, 55487)>

### Save gene matrix

In [17]:
GENE_PATH_DICT = {}


gene_df_path = result_dir / f'{page_name}.{mc_type}.gene.tsv'

if not gene_df_path.exists():
    _this_da = gene_mcds.sel(gene=gene_meta.index).squeeze()['gene_da']
    # gene_df should be gene by cell, row is gene
    gene_df = _this_da.to_pandas().T
    gene_df.columns.name = ''
    gene_df.index = gene_df.index.map(lambda i: f'{i}|{gene_meta.loc[i, "gene_name"]}')
    gene_df.to_csv(gene_df_path, sep='\t', float_format='%.2f')
    GENE_PATH_DICT[mc_type] = gene_df_path
                                      
    del gene_df
    del gene_mcds

## Conf file

In [18]:
default_config_dict = {
    'page_name': page_name + mc_type,
    'page_priority': 10,
    'page_tags': [],
    'page_short_label': page_name + ' ' + mc_type,
    'gene_matrix_path': gene_df_path.absolute(),
    'cell_meta_path': CELL_META_PATH,
    'categorical_cols': categorical_cols,
    'coord_file_list': COORD_PATH_LIST,
    'default_cluster_field': default_cluster_col,
    'default_label_field': default_label_col,
    'gene_value_unit': f'Norm {mc_type}%',
    'color_path': COLOR_PATH,
    'radius': radius,
    'quick_gene_path': quick_gene_path
}

In [19]:
with open('cellbrowser_template.conf') as f, open(result_dir / '__cellbrowser.conf', 'w') as out_f:
    config_str_template = f.read()
    config_str = config_str_template.format(**default_config_dict)
    
    # add collection
    config_str += "\ncollections = ['CEMBA-RostralBrain']"
    
    out_f.write(config_str)

In [20]:
print(config_str)

# --------- REQUIRED SETTINGS --------------

name = "CEMBA-RostralBrainCGN"

priority = 10

tags = []

shortLabel="CEMBA-RostralBrain CGN"

exprMatrix="/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/CellBrowser/CEMBA-RostralBrain_CGN/CEMBA-RostralBrain.CGN.gene.tsv"

geneIdType="auto"

meta="/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/CellBrowser/CEMBA-RostralBrain_CGN/CEMBA-RostralBrain.cell_meta.tsv"

# we try to auto-detect the field type of fields in the meta data.
# Sometimes, this doesn't work, e.g. when your cluster ID is a numer
# or your C1 chip ID is a number, but you don't want them binned, you want
# to treat as if they were categories
enumFields = ['Row384', 'Col384', 'Row96', 'Col96', 'Pos96', 'FACS_Date', 'Plate', 'Region', 'Slice', 'index_name', 'uid', 'CellClass', 'MajorType', 'SubTypeMajorRegion', 'SubRegion']

# tsv files with coordinates of every sample in format <sampleId, x, y>
# first the name of the file, then a human read